In [2]:
import pandas as pd 
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
df.head(10)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
5,ATVI,Activision Blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",2015-08-31,718877,2008
6,ADM,ADM,reports,Consumer Staples,Agricultural Products,"Chicago, Illinois",1981-07-29,7084,1902
7,ADBE,Adobe Inc.,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
8,ADP,ADP,reports,Information Technology,Data Processing & Outsourced Services,"Roseland, New Jersey",1981-03-31,8670,1949
9,AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",2015-07-09,1158449,1932


In [18]:
df[df.CIK == 1841666]['Date first added'][df[df.CIK == 1841666]['Date first added'].index[0]]

'1997-07-28'

In [15]:
# import our libraries
import requests
from bs4 import BeautifulSoup
import time 
import warnings
warnings.filterwarnings("ignore")
# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

details_dictionary = {'ticker':[],'Date_of_listing':[],'Filing_date':[],'Filing_url':[]}
for cik in range(430,450):
    ticker = df[df.CIK == df.CIK[cik]]['Symbol'][df[df.CIK == df.CIK[cik]]['Symbol'].index[0]]
    date_of_listing = df[df.CIK == df.CIK[cik]]['Date first added'][df[df.CIK == df.CIK[cik]]['Date first added'].index[0]]
    print(ticker)
    print(date_of_listing)
    # initialize our list for storage
    master_list_xml = []
    for index in range(0,400,100):
        try:
            # define our parameters dictionary
            param_dict = {'action':'getcompany',
                          'CIK':f'{df.CIK[cik]}',
                          'type':'8-k',
                          'datea':'20101201',
                          'dateb':'20200320',
                          'owner':'exclude',
                          'start':f'{index}',
                          'output':'atom',
                          'count':'100'}

            headers = {'User-Agent':"mshrimandhar@gmail.com"}

            # request the url, and then parse the response.
            response = requests.get(url = endpoint,headers = headers, params = param_dict)
            soup = BeautifulSoup(response.content, 'lxml')
            
            # parsing all entry tags
            for entry in soup.find_all('entry'):
                accession_num = entry.find('accession-number').text
                # create a new dictionary
                entry_dict = {}
                entry_dict[accession_num] = {}

                # store category info
                category_info = entry.find('category')
                entry_dict[accession_num]['category_info'] = {}
                entry_dict[accession_num]['category_info']['label'] = category_info['label']
                entry_dict[accession_num]['category_info']['scheme'] = category_info['scheme']
                entry_dict[accession_num]['category_info']['term'] = category_info['term']

                # store the file info 
                entry_dict[accession_num]['file_info'] = {}
                try:
                    entry_dict[accession_num]['file_info']['act'] = entry.find('act').text
                except:
                    entry_dict[accession_num]['file_info']['act'] = ''


                entry_dict[accession_num]['file_info']['file_number'] = entry.find('file-number').text    
                entry_dict[accession_num]['file_info']['file_number_href'] = entry.find('file-number-href').text
                entry_dict[accession_num]['file_info']['filing_date'] = entry.find('filing-date').text    
                entry_dict[accession_num]['file_info']['filing_href'] = entry.find('filing-href').text    
                entry_dict[accession_num]['file_info']['filing_type'] = entry.find('filing-type').text    
                entry_dict[accession_num]['file_info']['form_number'] = entry.find('film-number').text        
                entry_dict[accession_num]['file_info']['form_name'] = entry.find('form-name').text
                entry_dict[accession_num]['file_info']['file_size'] = entry.find('size').text
                try:
                    entry_dict[accession_num]['file_info']['xbrl_href'] = entry.find('xbrl_href').text

                except:
                    entry_dict[accession_num]['file_info']['xbrl_href'] = ''

                # store extra info
                entry_dict[accession_num]['request_info'] = {}
                entry_dict[accession_num]['request_info']['link'] = entry.find('link')['href']
                entry_dict[accession_num]['request_info']['title'] = entry.find('title').text
                entry_dict[accession_num]['request_info']['last_update'] = entry.find('updated').text

                # store in master list
                master_list_xml.append(entry_dict)
            
        except Exception as e:
            print(f"Exception occured because \n {e} \n")

    print('='*70,f'Master list length for {df.CIK[cik]} is',len(master_list_xml))
    count = 0
    for filing in master_list_xml:
        print(filing[list(filing.keys())[0]]['file_info']['filing_href'])
        response_ex99 = requests.get(url = filing[list(filing.keys())[0]]['file_info']['filing_href'],headers = headers)
        dataframes = pd.read_html(response_ex99.text)
        print(count,'-->',"="*100)
        try:
            print('/'.join(filing[list(filing.keys())[0]]['file_info']['filing_href'].split('/')[:-1])+'/'+dataframes[0][(dataframes[0].Type == 'EX-99.1') | (dataframes[0].Type == 'EX-99')]['Document'].values[0])
            details_dictionary['Filing_url'].append('/'.join(filing[list(filing.keys())[0]]['file_info']['filing_href'].split('/')[:-1])+'/'+dataframes[0][(dataframes[0].Type == 'EX-99.1') | (dataframes[0].Type == 'EX-99')]['Document'].values[0])
            details_dictionary['Filing_date'].append(filing[list(filing.keys())[0]]['file_info']['filing_date'])
            details_dictionary['ticker'].append(ticker)
            details_dictionary['Date_of_listing'].append(date_of_listing)
        except Exception as e:
            print(f"Not available because {e}")
        print('Request Successful')
        print(response.status_code)
        time.sleep(0.2)
        count+=1
detail_df = pd.DataFrame(details_dictionary)
detail_df.to_csv(f'{cik+1}.csv',index=False)


SIVB
2018-03-19
====================================================================== Master list length for 719739 is 130
https://www.sec.gov/Archives/edgar/data/719739/000071973920000011/0000719739-20-000011-index.htm
0 --> ====================================================================================================
Not available because index 0 is out of bounds for axis 0 with size 0
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973920000008/0000719739-20-000008-index.htm
1 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973920000008/q419irpresentation.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973920000004/0000719739-20-000004-index.htm
2 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973920

26 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973918000042/q318earningsrelease_exhibi.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973918000035/0000719739-18-000035-index.htm
27 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973918000035/q218irpresentation.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973918000032/0000719739-18-000032-index.htm
28 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973918000032/q218earningsrelease_exhibi.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973918000019/0000719739-18-000019-index.htm
29 --> ==========================

52 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973916000106/q316earningsrelease_exhibi.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973916000092/0000719739-16-000092-index.htm
53 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973916000092/svb2016q2ir.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973916000090/0000719739-16-000090-index.htm
54 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973916000090/q216earningsrelease_exhibi.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973916000082/0000719739-16-000082-index.htm
55 --> =================================

79 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973914000029/q214earningsrelease_exhibi.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000119312514204708/0001193125-14-204708-index.htm
80 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000119312514204708/d722212dex991.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973914000025/0000719739-14-000025-index.htm
81 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000071973914000025/svb2014q1irslidesfinal.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000071973914000018/0000719739-14-000018-index.htm
82 --> ===================================

105 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/719739/000119312512222349/d349964dex991.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000119312512196097/0001193125-12-196097-index.htm
106 --> ====================================================================================================
Not available because index 0 is out of bounds for axis 0 with size 0
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000119312512186020/0001193125-12-186020-index.htm
107 --> ====================================================================================================
Not available because index 0 is out of bounds for axis 0 with size 0
Request Successful
200
https://www.sec.gov/Archives/edgar/data/719739/000119312512186005/0001193125-12-186005-index.htm
108 --> =================================================================================

...

19 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/916365/000091636519000078/ex991-q12019earningsrelease.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636519000038/0000916365-19-000038-index.htm
20 --> ====================================================================================================
Not available because index 0 is out of bounds for axis 0 with size 0
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636519000020/0000916365-19-000020-index.htm
21 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/916365/000091636519000020/a020719-ex991electionofdir.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636519000017/0000916365-19-000017-index.htm
22 --> ============================================

45 --> ====================================================================================================
Not available because index 0 is out of bounds for axis 0 with size 0
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636517000077/0000916365-17-000077-index.htm
46 --> ====================================================================================================
Not available because index 0 is out of bounds for axis 0 with size 0
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636517000056/0000916365-17-000056-index.htm
47 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/916365/000091636517000056/a05092017tsco-ex991.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636517000053/0000916365-17-000053-index.htm
48 --> ===============================================================================

https://www.sec.gov/Archives/edgar/data/916365/000091636516000107/0000916365-16-000107-index.htm
72 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/916365/000091636516000107/q42015businessupdate.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636516000105/0000916365-16-000105-index.htm
73 --> ====================================================================================================
Not available because index 0 is out of bounds for axis 0 with size 0
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636515000101/0000916365-15-000101-index.htm
74 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/916365/000091636515000101/a11032015tsco-ex991.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636515000099

98 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/916365/000091636514000062/a022414-repurchaseplanex991.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636514000050/0000916365-14-000050-index.htm
99 --> ====================================================================================================
Not available because index 0 is out of bounds for axis 0 with size 0
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636514000032/0000916365-14-000032-index.htm
100 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/916365/000091636514000032/a02072014-electionofdirect.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636514000030/0000916365-14-000030-index.htm
101 --> ==========================================

124 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/916365/000091636512000044/pressrelease.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636512000041/0000916365-12-000041-index.htm
125 --> ====================================================================================================
https://www.sec.gov/Archives/edgar/data/916365/000091636512000041/pressrelease.htm
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636512000031/0000916365-12-000031-index.htm
126 --> ====================================================================================================
Not available because index 0 is out of bounds for axis 0 with size 0
Request Successful
200
https://www.sec.gov/Archives/edgar/data/916365/000091636512000013/0000916365-12-000013-index.htm
127 --> =====================================================================

In [13]:
detail_df = pd.DataFrame(details_dictionary)
detail_df.to_csv(f'{cik+1}.csv',index=False)

In [14]:
print(cik)

429


In [ ]:
import pandas as pd
import time 
count = 0
for filing in master_list_xml:
    print(filing[list(filing.keys())[0]]['file_info']['filing_href'])
    response_ex99 = requests.get(url = filing[list(filing.keys())[0]]['file_info']['filing_href'],headers = headers)
    dataframes = pd.read_html(response_ex99.text)
    print(count,'-->',"="*100)
    try:
        print('/'.join(filing[list(filing.keys())[0]]['file_info']['filing_href'].split('/')[:-1])+'/'+dataframes[0][(dataframes[0].Type == 'EX-99.1') | (dataframes[0].Type == 'EX-99')]['Document'].values[0])
        print(filing[list(filing.keys())[0]]['file_info']['filing_date'])
    except Exception as e:
        print(f"Not available because {e}")

    print('Request Successful')
    print(response.status_code)
    time.sleep(0.1)
    count+=1

In [152]:
text = 'https://www.sec.gov/Archives/edgar/data/1267238/000119312516658147/0001193125-16-658147-index.htm'
text_to_append = 'd230270dex991.htm'
print('/'.join(text.split('/')[:-1])+'/'+text_to_append)

https://www.sec.gov/Archives/edgar/data/1267238/000119312516658147/d230270dex991.htm


In [91]:
master_list_xml[0]

{'0001193125-17-288464': {'category_info': {'label': 'form type',
   'scheme': 'https://www.sec.gov/',
   'term': '8-K'},
  'file_info': {'act': '34',
   'file_number': '001-37845',
   'file_number_href': 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=001-37845&owner=exclude&count=10',
   'filing_date': '2017-09-19',
   'filing_href': 'https://www.sec.gov/Archives/edgar/data/789019/000119312517288464/0001193125-17-288464-index.htm',
   'filing_type': '8-K',
   'form_number': '171092653',
   'form_name': 'Current report',
   'file_size': '23 KB',
   'xbrl_href': ''},
  'request_info': {'link': 'https://www.sec.gov/Archives/edgar/data/789019/000119312517288464/0001193125-17-288464-index.htm',
   'title': '8-K  - Current report',
   'last_update': '2017-09-19T17:14:03-04:00'}}}

In [127]:
response_ex99 = requests.get(url = 'https://www.sec.gov/Archives/edgar/data/1267238/000119312516658147/0001193125-16-658147-index.htm',headers = headers)
soup_ex99 = BeautifulSoup(response_ex99.content, 'lxml')

# Let the user know it was successful.
print('Request Successful')
print(response.status_code)
print(response.url)

Request Successful
200
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1267238&type=8-k&datea=20101201&owner=exclude&start=&output=atom&count=100


In [128]:
import pandas as pd
dfs = pd.read_html(response_ex99.text)

In [129]:
df = dfs[0]
df

,Seq,Description,Document,Type,Size
0,1.0,FORM 8-K,d230270d8k.htm,8-K,14082
1,2.0,EX-99.1,d230270dex991.htm,EX-99.1,160635
2,3.0,GRAPHIC,g2302701.jpg,GRAPHIC,7518
3,NaN,Complete submission text file,0001193125-16-658147.txt,NaN,186199


In [143]:
df_ex_99 = df[df.Type == 'EX-99.1']['Document'].values[0]
df_ex_99

'd230270dex991.htm'

In [25]:
# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

# define our parameters dictionary
param_dict = {'action':'getcompany',
              'CIK':'1265107',
              'dateb':'20190101',
              'owner':'exclude',
              'start':'',
              'output':'atom',
              'count':'100'}

# request the url, and then parse the response.
response = requests.get(url = endpoint, params = param_dict)
soup = BeautifulSoup(response.content, 'lxml')

# find the link that will take us to the next page
links = soup.find_all('link',{'rel':'next'})

# while there is still a next page
while soup.find_all('link',{'rel':'next'}) != []:

    # grab the link
    next_page_link = links[0]['href']  
    
    print('-'*100)
    print(next_page_link)
    
    # request the next page
    response = requests.get(url = next_page_link)
    soup = BeautifulSoup(response.content, 'lxml')
    
    # see if there is a next link
    links = soup.find_all('link',{'rel':'next'})

In [26]:
print(links)

[]
